# BlockCluster EDA | Kyle McLester
The data for this project is found on [Kaggle](https://www.kaggle.com/rounakbanik/the-movies-dataset). Our goal for this portion of the project is to perform a basic exploratory analysis of the data and report our findings

## Importing Dependencies

In [1]:
from networkx.algorithms.distance_measures import radius
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import operator
import os
import numpy as np
from pathlib import Path
import plotly.express as go
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# set app threads to 16 (max of my system)
os.environ["NUMEXPR_MAX_THREADS"] = "16"
os.environ["NUMEXPR_NUM_THREADS"] = "16"
pd.options.mode.chained_assignment = None
# force matplotlib graphs to be same shape
plt.rcParams.update({"figure.autolayout": True})

## Loading Data

In [2]:
links = pd.read_csv('data/movielens/100k/links.csv', sep=',', encoding='latin-1')
movies = pd.read_csv('data/movielens/100k/movies.csv', sep=',', encoding='latin-1')
ratings = pd.read_csv('data/movielens/100k/ratings.csv', sep=',', encoding='latin-1', usecols=['userId','movieId','rating'])
tags = pd.read_csv('data/movielens/100k/tags.csv', sep=',', encoding='latin-1', usecols=['userId','movieId','tag'])

## Exploring Data

In [3]:
print(5*'*'+' COLUMN LABELS '+5*'*')
print('links: {}'.format(list(links.columns)))
print('movies: {}'.format(list(movies.columns)))
print('ratings: {}'.format(list(ratings.columns)))
print('tags: {}'.format(list(tags.columns)))

***** COLUMN LABELS *****
links: ['movieId', 'imdbId', 'tmdbId']
movies: ['movieId', 'title', 'genres']
ratings: ['userId', 'movieId', 'rating']
tags: ['userId', 'movieId', 'tag']


In [4]:
print(5*'*'+' DATA SHAPE '+5*'*')
print('links: {}'.format(links.shape))
print('movies: {}'.format(movies.shape))
print('ratings: {}'.format(ratings.shape))
print('tags: {}'.format(tags.shape))

***** DATA SHAPE *****
links: (9742, 3)
movies: (9742, 3)
ratings: (100836, 3)
tags: (3683, 3)


In [5]:
print(5*'*'+' RATING SUMMARY STATS '+5*'*')
ratings['rating'].describe()

***** RATING SUMMARY STATS *****


count    100836.000000
mean          3.501557
std           1.042529
min           0.500000
25%           3.000000
50%           3.500000
75%           4.000000
max           5.000000
Name: rating, dtype: float64

In [5]:
print(5*'*'+' NULL VALUE COUNTS '+5*'*')
print('links\n{}\n'.format(links.isnull().sum()))
print('movies\n{}\n'.format(movies.isnull().sum()))
print('ratings\n{}\n'.format(ratings.isnull().sum()))
print('tags\n{}\n'.format(tags.isnull().sum()))

***** NULL VALUE COUNTS *****
links
movieId    0
imdbId     0
tmdbId     8
dtype: int64

movies
movieId    0
title      0
genres     0
dtype: int64

ratings
userId     0
movieId    0
rating     0
dtype: int64

tags
userId     0
movieId    0
tag        0
dtype: int64



In [6]:
print(5*'*'+' REMOVE ROWS W/NULL VALUES ' + 5*'*')
links = links.dropna()
links.isnull().sum()

***** REMOVE ROWS W/NULL VALUES *****


movieId    0
imdbId     0
tmdbId     0
dtype: int64

In [7]:
print(5*'*'+' FIRST 5 ROWS '+5*'*')
print('links\n{}\n'.format(links.head()))
print('movies\n{}\n'.format(movies.head()))
print('ratings\n{}\n'.format(ratings.head()))
print('tags\n{}\n'.format(tags.head()))

***** FIRST 5 ROWS *****
links
   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0

movies
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

ratings
   userId  movieId  rating
0       1        1     4.0
1       1        3     4.0
2       1        6     4.0
3       1       47     5.0
4       1       50     5.0

tags
   userId  movieId   

In [8]:
# break up movies['genre'] into a string
movies['genres'] = movies['genres'].str.split('|')
movies['genres'] = movies['genres'].fillna('').astype('str')

In [9]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy..."
1,2,Jumanji (1995),"['Adventure', 'Children', 'Fantasy']"
2,3,Grumpier Old Men (1995),"['Comedy', 'Romance']"
3,4,Waiting to Exhale (1995),"['Comedy', 'Drama', 'Romance']"
4,5,Father of the Bride Part II (1995),['Comedy']


In [10]:
# extract year from title column
movies['year'] = [re.search('(\d){4}', x) for x in movies['title']]
movies = movies.dropna()
movies['year'] = [re.search('(\d){4}', x).group() for x in movies['title']]
movies = movies.replace(to_replace='\((.*)\)', value='', regex=True)
movies = movies.replace(to_replace=' +$', value='', regex=True)
movies['year'] = movies['year'].astype('int')

In [11]:
movies.head()

,movieId,title,genres,year
0,1,Toy Story,"['Adventure', 'Animation', 'Children', 'Comedy...",1995
1,2,Jumanji,"['Adventure', 'Children', 'Fantasy']",1995
2,3,Grumpier Old Men,"['Comedy', 'Romance']",1995
3,4,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",1995
4,5,Father of the Bride Part II,['Comedy'],1995


In [12]:
# remove title that were missing a year or are recorded after this study
movies = movies[movies['year']<=2018]

## Recommendation based on Genre

In [13]:
tf = TfidfVectorizer(analyzer='word',
                     ngram_range=(1,2),
                     min_df=0,
                     stop_words='english')

In [14]:
tfidf_matrix = tf.fit_transform(movies['genres'])
print('tfidf_matrix\nshape: {}'.format(tfidf_matrix.shape))

tfidf_matrix
shape: (9724, 174)


In [15]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim[:4, :4]

array([[1.        , 0.31378594, 0.06107542, 0.05268521],
       [0.31378594, 1.        , 0.        , 0.        ],
       [0.06107542, 0.        , 1.        , 0.3516825 ],
       [0.05268521, 0.        , 0.3516825 , 1.        ]])

In [16]:
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

In [17]:
def genre_recommendations(title, num_titles):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num_titles+1]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [18]:
genre_recommendations('Tomb Raider', 10)

1516               Flight of the Navigator
4274    Journey to the Center of the Earth
5004                             Explorers
5396                            D.A.R.Y.L.
6436                       Last Mimzy, The
6862                         City of Ember
9016        Doctor Who: The Waters of Mars
1506                       Black Hole, The
1554            Return from Witch Mountain
2803                            Titan A.E.
Name: title, dtype: object

In [19]:
genre_recommendations('Ant-Man and the Wasp', 10)

2861                                        Space Cowboys
3069                                           Innerspace
3927                        Adventures of Pluto Nash, The
5259                                Last Starfighter, The
6754    Indiana Jones and the Kingdom of the Crystal S...
7052                                     Land of the Lost
8691                                             Deadpool
9731                                              Gintama
560                                            Barbarella
969                                    Back to the Future
Name: title, dtype: object

## Recommendation based on Title

In [24]:
tf = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=0,
                     stop_words='english')

In [25]:
tfidf_matrix =tf.fit_transform(movies['title'])
tfidf_matrix.shape

(9724, 15855)

In [26]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim[:4, :4]

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [27]:
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

In [28]:
def title_recommendations(title, num_titles):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num_titles+1]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [29]:
title_recommendations('Tomb Raider', 10)

9433                         Rogue One: A Star Wars Story
567                                                  Solo
6823                            Star Wars: The Clone Wars
9645                             Star Wars: The Last Jedi
8908                        The Star Wars Holiday Special
7367    Empire of Dreams: The Story of the 'Star Wars'...
1979            Star Wars: Episode I - The Phantom Menace
898        Star Wars: Episode V - The Empire Strikes Back
1570                                           L.A. Story
2227                                     Story of Us, The
Name: title, dtype: object

In [28]:
title_recommendations('Toy Story', 10)

2355           Toy Story 2
7355           Toy Story 3
3595              Toy, The
1570            L.A. Story
2227      Story of Us, The
4156       Story of O, The
4089          Toy Soldiers
3187            Love Story
2110    Christmas Story, A
4047           Ghost Story
Name: title, dtype: object

## Average Rating Given by Each User

In [30]:
merged_df = ratings.join(movies.set_index('movieId'), on='movieId')

In [31]:
merged_df[merged_df['userId']==1]

,userId,movieId,rating,title,genres,year
0,1,1,4.0,Toy Story,"['Adventure', 'Animation', 'Children', 'Comedy...",1995.0
1,1,3,4.0,Grumpier Old Men,"['Comedy', 'Romance']",1995.0
2,1,6,4.0,Heat,"['Action', 'Crime', 'Thriller']",1995.0
3,1,47,5.0,Seven,"['Mystery', 'Thriller']",1995.0
4,1,50,5.0,"Usual Suspects, The","['Crime', 'Mystery', 'Thriller']",1995.0
...,...,...,...,...,...,...
227,1,3744,4.0,Shaft,"['Action', 'Crime', 'Thriller']",2000.0
228,1,3793,5.0,X-Men,"['Action', 'Adventure', 'Sci-Fi']",2000.0
229,1,3809,4.0,What About Bob?,['Comedy'],1991.0
230,1,4006,4.0,Transformers: The Movie,"['Adventure', 'Animation', 'Children', 'Sci-Fi']",1986.0


In [32]:
user_avg_ratings = pd.DataFrame(columns=['userId','avg_rating'])
userId = []
avg_rating = []
for user in merged_df['userId'].unique():
    userId.append(user)
    avg_rating.append(merged_df[merged_df['userId']==user]['rating'].sum()/len(merged_df[merged_df['userId']==user]))
user_avg_ratings['userId'] = userId
user_avg_ratings['avg_rating'] = avg_rating

In [33]:
user_avg_ratings = user_avg_ratings.set_index('userId')

In [34]:
user_avg_ratings.describe()

,avg_rating
count,610.000000
mean,3.657222
std,0.480635
min,1.275000
25%,3.360000
50%,3.694385
75%,3.997500
max,5.000000


In [36]:
user_avg_ratings

,avg_rating
userId,
1,4.366379
2,3.948276
3,2.435897
4,3.555556
5,3.636364
...,...
606,3.657399
607,3.786096
608,3.134176


## Average Rating for Each Movie (Naive)

In [54]:
avg_movie_ratings = pd.DataFrame(columns=['movieId','avg_rating'])
movieId = []
avg_rating = []
vote_count = []
for movie in merged_df['movieId'].unique():
    movieId.append(movie)
    avg_rating.append(merged_df[merged_df['movieId']==movie]['rating'].sum()/len(merged_df[merged_df['movieId']==movie]))
    vote_count.append(merged_df[merged_df['movieId']==movie]['rating'].count())
avg_movie_ratings['movieId'] = movieId
avg_movie_ratings['avg_rating'] = avg_rating
avg_movie_ratings['vote_count'] = vote_count

In [55]:
avg_movie_ratings = avg_movie_ratings.join(movies.set_index('movieId'), on='movieId')
avg_movie_ratings = avg_movie_ratings.drop(['genres','year'], axis=1)
avg_movie_ratings = avg_movie_ratings.dropna()

In [61]:
avg_movie_ratings.head()

,movieId,avg_rating,vote_count,title
0,1,3.920930,215,Toy Story
1,3,3.259615,52,Grumpier Old Men
2,6,3.946078,102,Heat
3,47,3.975369,203,Seven
4,50,4.237745,204,"Usual Suspects, The"


In [56]:
# 10 lowest rated movies
avg_movie_ratings.sort_values(by=['avg_rating'])[:10]

,movieId,avg_rating,vote_count,title
8941,7114,0.5,1,"Collector, The"
8859,122246,0.5,1,Tooth Fairy 2
8531,8632,0.5,1,Secret Society
7822,82095,0.5,1,Skyline
9298,136297,0.5,1,Mortal Kombat: The Journey Begins
6513,54934,0.5,1,"Brothers Solomon, The"
6518,67799,0.5,1,The Butterfly Effect 3: Revelations
9610,61818,0.5,1,"Crow, The: Wicked Prayer"
6535,134528,0.5,1,Aloha
6539,138798,0.5,1,Joe Dirt 2: Beautiful Loser


In [57]:
# 10 highest rated movies
avg_movie_ratings.sort_values(by=['avg_rating'], ascending=False)[:10]

,movieId,avg_rating,vote_count,title
4275,4788,5.0,1,Moscow Does Not Believe in Tears
8685,120130,5.0,1,Into the Forest of Fireflies' Light
7102,6021,5.0,1,"American Friend, The"
5572,5607,5.0,1,Son of the Bride
4277,5888,5.0,1,Brother
8557,71268,5.0,1,Tyler Perry's I Can Do Bad All by Myself
8558,93320,5.0,1,Trailer Park Boys
4278,5889,5.0,1,"Cruel Romance, A"
8580,467,5.0,1,Live Nude Girls
8581,876,5.0,1,Supercop 2


In [62]:
avg_movie_ratings['avg_rating'].describe()

count    9706.000000
mean        3.261957
std         0.869360
min         0.500000
25%         2.800000
50%         3.416667
75%         3.909773
max         5.000000
Name: avg_rating, dtype: float64

## IMDB Weighted Rating

In [66]:
C = avg_movie_ratings['avg_rating'].mean()
m = avg_movie_ratings['vote_count'].quantile(0.9)
avg_copy = avg_movie_ratings.copy().loc[avg_movie_ratings['vote_count']>=m]

In [67]:
avg_copy

,movieId,avg_rating,vote_count,title
0,1,3.920930,215,Toy Story
1,3,3.259615,52,Grumpier Old Men
2,6,3.946078,102,Heat
3,47,3.975369,203,Seven
4,50,4.237745,204,"Usual Suspects, The"
...,...,...,...,...
3350,8807,3.455882,34,Harold and Kumar Go to White Castle
3384,45720,3.558824,34,"Devil Wears Prada, The"
3490,94959,3.775862,29,Moonrise Kingdom
3498,1172,4.161765,34,Cinema Paradiso


In [68]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['avg_rating']
    return (v/(v+m)*R) + (m/(m/v)*C)

In [69]:
avg_copy['score'] = avg_copy.apply(weighted_rating, axis=1)

In [71]:
avg_copy.head()

,movieId,avg_rating,vote_count,title,score
0,1,3.920930,215,Toy Story,704.804189
1,3,3.259615,52,Grumpier Old Men,171.767325
2,6,3.946078,102,Heat,335.839752
3,47,3.975369,203,Seven,665.685932
4,50,4.237745,204,"Usual Suspects, The",669.181617


In [75]:
avg_copy.to_csv('weighted_ratings.csv',sep=',',encoding='utf-8', index=False)

In [72]:
avg_copy.sort_values(by='score',ascending=False, inplace=True)
avg_copy.head(20)

,movieId,avg_rating,vote_count,title,score
20,356,4.164134,329,Forrest Gump,1077.032111
232,318,4.429022,317,"Shawshank Redemption, The",1038.121710
16,296,4.197068,307,Pulp Fiction,1005.278531
34,593,4.161290,279,"Silence of the Lambs, The",913.880073
166,2571,4.192446,278,"Matrix, The",910.645310
15,260,4.231076,251,Star Wars: Episode IV - A New Hope,822.571308
26,480,3.750000,238,Jurassic Park,779.713650
7,110,4.031646,237,Braveheart,776.703087
478,589,3.970982,224,Terminator 2: Judgment Day,734.222154
28,527,4.225000,220,Schindler's List,721.393660


## Creating User Profiles

In [76]:
profiles = {}

for user in merged_df['userId'].unique():
    username = 'user_' + str(user)
    
    temp = merged_df[merged_df['userId']==user]
    
    titles = temp['title']
    ratings = temp['rating']
    ratings = list(ratings)
    
    movie_list = {}
    for i, title in enumerate(titles):
        movie_list[title] = ratings[i]
        
    profiles[username] = movie_list

In [80]:
import json

In [82]:
with open('profiles.json','w') as outfile:
    json.dump(profiles, outfile,indent=4)

In [83]:
def get_common_movies(user_A, user_B):
    return [movie for movie in profiles[user_A] if movie in profiles[user_B]]

In [84]:
# number of available user profiles
len(profiles)

610

In [85]:
get_common_movies('user_234','user_610')

['Toy Story',
 'Batman Forever',
 'Dumb & Dumber',
 'Star Wars: Episode IV - A New Hope',
 'Ace Ventura: Pet Detective',
 'Speed',
 'Jurassic Park',
 'Batman',
 'Wizard of Oz, The',
 'Die Hard',
 'E.T. the Extra-Terrestrial',
 'Star Wars: Episode V - The Empire Strikes Back',
 'Raiders of the Lost Ark',
 'Aliens',
 'Star Wars: Episode VI - Return of the Jedi',
 'Alien',
 'Army of Darkness',
 'Indiana Jones and the Last Crusade',
 'Face/Off',
 'As Good as It Gets',
 'Labyrinth',
 'Lethal Weapon',
 'Saving Private Ryan',
 'Dark Crystal, The',
 'NeverEnding Story, The',
 "Bug's Life, A",
 'Matrix, The',
 'Star Wars: Episode I - The Phantom Menace',
 'RoboCop',
 'Toy Story 2',
 'Beach, The',
 'Teenage Mutant Ninja Turtles',
 'Flintstones in Viva Rock Vegas, The',
 'Shanghai Noon',
 'Starman',
 'Chicken Run',
 'X-Men',
 "Charlie's Angels",
 'Cast Away',
 'Memento',
 'Spy Kids',
 'Mummy Returns, The',
 'Shrek',
 'Jurassic Park III']

In [86]:
# get reviews from common movies
def get_reviews(user_A, user_B):
    common_movies = get_common_movies(user_A, user_B)
    return [(profiles[user_A][movie], profiles[user_B][movie]) for movie in common_movies]

In [87]:
get_reviews('user_70','user_391')

[(4.0, 4.0),
 (4.5, 5.0),
 (4.0, 5.0),
 (5.0, 5.0),
 (5.0, 5.0),
 (5.0, 5.0),
 (4.5, 5.0),
 (5.0, 5.0),
 (5.0, 5.0),
 (5.0, 4.0),
 (5.0, 4.0),
 (4.5, 4.0),
 (4.0, 4.0),
 (3.5, 4.0),
 (4.5, 3.0),
 (4.0, 4.0),
 (4.0, 4.0),
 (3.5, 4.0),
 (4.0, 4.0),
 (4.5, 4.0)]

In [88]:
# get reviews from common movies
def get_reviews_detailed(user_A, user_B):
    common_movies = get_common_movies(user_A, user_B)
    reviews = [(profiles[user_A][movie], profiles[user_B][movie]) for movie in common_movies]
    result = zip(common_movies, reviews)
    return set(result)

In [89]:
get_reviews_detailed('user_70','user_391')

{('American Beauty', (4.0, 4.0)),
 ('Apollo 13', (4.0, 4.0)),
 ('Back to the Future', (4.0, 4.0)),
 ('Beautiful Mind, A', (4.5, 4.0)),
 ('Breakfast Club, The', (4.0, 4.0)),
 ('Cinema Paradiso', (5.0, 5.0)),
 ('Dead Poets Society', (5.0, 4.0)),
 ('E.T. the Extra-Terrestrial', (4.5, 5.0)),
 ("Ferris Bueller's Day Off", (3.5, 4.0)),
 ('Forrest Gump', (4.0, 5.0)),
 ('Godfather, The', (5.0, 5.0)),
 ('Good Will Hunting', (4.5, 3.0)),
 ("It's a Wonderful Life", (5.0, 5.0)),
 ("One Flew Over the Cuckoo's Nest", (5.0, 5.0)),
 ("Schindler's List", (5.0, 5.0)),
 ('Shawshank Redemption, The', (4.5, 5.0)),
 ('Shrek', (4.0, 4.0)),
 ('Stand by Me', (4.5, 4.0)),
 ('Star Wars: Episode VI - Return of the Jedi', (5.0, 4.0)),
 ('Truman Show, The', (3.5, 4.0))}

In [90]:
def euclidean_distance(points):
    squared_diff = [(point[0] - point[1]) ** 2 for point in points]
    summed_squared_diffs = sum(squared_diff)
    distance = np.sqrt(summed_squared_diffs)
    return distance

In [91]:
def similarity(reviews):
    return 1/(1+euclidean_distance(reviews))

In [92]:
def get_user_similarity(user_A, user_B):
    reviews = get_reviews(user_A, user_B)
    return similarity(reviews)

In [95]:
get_user_similarity('user_1','user_86')

0.21239994067041815

In [96]:
def recommend_movies(user, num_suggestions):
    similarity_scores = [(get_user_similarity(user, other), other) for other in profiles if other != user]
    # get sim scores for all users
    similarity_scores.sort()
    similarity_scores.reverse()
    # similarity_scores = similarity_scores[0:num_suggestions]
    
    recommendations = {}
    for similarity, other in similarity_scores:
        reviewed = profiles[other]
        for movie in reviewed:
            if movie not in profiles[user]:
                weight = similarity * reviewed[movie]
                if movie in recommendations:
                    sim, weights = recommendations[movie]
                    recommendations[movie] = (sim + similarity, weights + [weight])
                else:
                    recommendations[movie] = (similarity, [weight])
                    
    for recommendation in recommendations:
        similarity, movie = recommendations[recommendation]
        recommendations[recommendation] = sum(movie) / similarity
        
    sorted_recommendations = sorted(recommendations.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_recommendations[:num_suggestions]

In [97]:
recommend_movies('user_203', 10)

[('Jonah Who Will Be 25 in the Year 2000', 5.000000000000001),
 ('Come and See', 5.000000000000001),
 ('Story of Women', 5.0),
 ('Passenger, The', 5.0),
 ('Marriage of Maria Braun, The', 5.0),
 ('61*', 5.0),
 ('Thin Line Between Love and Hate, A', 5.0),
 ('Animals are Beautiful People', 5.0),
 ('Woman Is a Woman, A', 5.0),
 ('Thousand Clowns, A', 5.0)]

In [279]:
recommend_movies('user_120', 10)

[('Particle Fever', 5.000000000000001),
 ('Heidi Fleiss: Hollywood Madam', 5.000000000000001),
 ('Lamerica', 5.0),
 ('Entertaining Angels: The Dorothy Day Story', 5.0),
 ('Story of Women', 5.0),
 ('Passenger, The', 5.0),
 ('Marriage of Maria Braun, The', 5.0),
 ('One I Love, The', 5.0),
 ('Laggies', 5.0),
 ('Delirium', 5.0)]

In [98]:
merged_df = merged_df.dropna(axis=0)

In [99]:
merged_df.head()

,userId,movieId,rating,title,genres,year
0,1,1,4.0,Toy Story,"['Adventure', 'Animation', 'Children', 'Comedy...",1995.0
1,1,3,4.0,Grumpier Old Men,"['Comedy', 'Romance']",1995.0
2,1,6,4.0,Heat,"['Action', 'Crime', 'Thriller']",1995.0
3,1,47,5.0,Seven,"['Mystery', 'Thriller']",1995.0
4,1,50,5.0,"Usual Suspects, The","['Crime', 'Mystery', 'Thriller']",1995.0


In [100]:
merged_df['year'] = [int(x) for x in merged_df['year']]

In [101]:
merged_df.head()

,userId,movieId,rating,title,genres,year
0,1,1,4.0,Toy Story,"['Adventure', 'Animation', 'Children', 'Comedy...",1995
1,1,3,4.0,Grumpier Old Men,"['Comedy', 'Romance']",1995
2,1,6,4.0,Heat,"['Action', 'Crime', 'Thriller']",1995
3,1,47,5.0,Seven,"['Mystery', 'Thriller']",1995
4,1,50,5.0,"Usual Suspects, The","['Crime', 'Mystery', 'Thriller']",1995


In [102]:
import plotly.express as px

In [103]:
merged_df = merged_df[merged_df['year']>=1915]

In [104]:
fig = px.bar(merged_df, merged_df['year'].unique(), merged_df['year'].value_counts(), title='Movies Produced by Year')

In [105]:
fig.show()

In [106]:
small_df = merged_df[0:round(len(merged_df)*0.01)]

In [107]:
small_df['userId'] = ['user_'+str(x) for x in small_df['userId']]

In [108]:
import networkx as nx

In [109]:
G = nx.Graph()

In [110]:
G = nx.from_pandas_edgelist(small_df, source='userId',target='title', edge_attr='rating')

In [111]:
from pyvis.network import Network

In [ ]:
net = Network(notebook=True)
net.from_nx(G)
net.inherit_edge_colors(True)
net.show('network_graph.html')

In [113]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
merged_df['genres_cat'] = encoder.fit_transform(merged_df['genres'])
merged_df['title_cat'] = encoder.fit_transform(merged_df['title'])
merged_df.head()

,userId,movieId,rating,title,genres,year,genres_cat,title_cat
0,1,1,4.0,Toy Story,"['Adventure', 'Animation', 'Children', 'Comedy...",1995,353,8545
1,1,3,4.0,Grumpier Old Men,"['Comedy', 'Romance']",1995,734,3522
2,1,6,4.0,Heat,"['Action', 'Crime', 'Thriller']",1995,260,3695
3,1,47,5.0,Seven,"['Mystery', 'Thriller']",1995,934,7235
4,1,50,5.0,"Usual Suspects, The","['Crime', 'Mystery', 'Thriller']",1995,787,8783


In [134]:
def predict_rating(user, title):
    similarity_scores = [(get_user_similarity(user, other), other) for other in profiles if other != user]
    # get sim scores for all users
    similarity_scores.sort()
    similarity_scores.reverse()
    
    recommendations = {}
    for similarity, other in similarity_scores:
        reviewed = profiles[other]
        for movie in reviewed:
            weight = similarity * reviewed[movie]
            if movie in recommendations:
                sim, weights = recommendations[movie]
                recommendations[movie] = (sim + similarity, weights + [weight])
            else:
                recommendations[movie] = (similarity, [weight])
                    
    for recommendation in recommendations:
        similarity, movie = recommendations[recommendation]
        recommendations[recommendation] = sum(movie) / similarity

    return recommendations[title]

In [135]:
predict_rating('user_1','Toy Story')

3.9972001085979056

In [140]:
for movie in profiles['user_1']:
    try:
        print(f'{movie} : {predict_rating("user_1",movie)}')
    except:
        pass

Toy Story : 3.9972001085979056
Grumpier Old Men : 3.385277489894013
Heat : 4.025073932981687
Seven : 4.075882463767784
Usual Suspects, The : 4.318454264952223
From Dusk Till Dawn : 3.5075135392341017
Bottle Rocket : 3.603532650064559
Braveheart : 4.129677429373077
Rob Roy : 3.4941203544029413
Canadian Bacon : 2.6628028666138572
Desperado : 3.592462500510135
Billy Madison : 3.5170013634991286
Clerks : 3.946175652096673
Dumb & Dumber : 3.1914582113205645
Ed Wood : 3.7113364769052795
Star Wars: Episode IV - A New Hope : 4.310235491294956
Pulp Fiction : 4.168676560532393
Stargate : 3.3542659908323773
Tommy Boy : 3.936019231490493
Clear and Present Danger : 3.657129449485551
Forrest Gump : 4.243630286380716
Jungle Book, The : 3.803288394796192
Mask, The : 3.270954004722741
Blown Away : 2.778289481613553
Dazed and Confused : 3.8431059420616744
Fugitive, The : 4.079834307198037
Jurassic Park : 3.801069861844889
Mrs. Doubtfire : 3.462957106246344
Schindler's List : 4.352932631731257
So I Marri